# Librairie

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.linear_model import BayesianRidge
import missingno as msno 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.display import Image


In [4]:
def txt_to_csv(path_src,path_dst):
    df = pd.read_csv(path_src, delim_whitespace=True, header=None)
    df.columns = ['mpg','cylinders quantitative','displacement','horsepower','weight','acceleration','model year','origin','car name']
    df.to_csv (path_dst, index=None)
    return df

In [5]:
def columns_na(df):
    null_columns=df.columns[df.isnull().any()]
    df[null_columns].isnull().sum()

    for column in null_columns:
        print ("La colonne ",column," a ", df[column].isnull().sum(), " valeurs manquantes, soit ", df[column].isnull().sum()/(len(df[column])*0.01),"% de valeurs manquantes")
    msno.matrix(df,color=(0, 0, 0.25),fontsize=15,)

In [6]:
def sup_values_columns(columns_name, pourcentage,dataset):
    dataset_modifie=dataset.copy()
    columns=dataset_modifie.columns
    if (sum(columns_name==columns) != 1):
        print("Colonnes n'ont existante")
    
    #nombre de valeur de la colonne
    Nbr_values=len(dataset_modifie[columns_name])
    
    #nombre de valeur a supprimer
    delete=int(Nbr_values*pourcentage/100)
    
    while(pd.isnull(dataset_modifie[columns_name]).sum()<delete):
        #selection de l'index
        index_delete=random.randrange(Nbr_values)      
        dataset_modifie[columns_name][index_delete]= np.NaN
        
    return dataset_modifie

In [7]:
def sup_values_datasete(pourcentage,dataset):
    dataset_modifie=dataset.copy()
    columns=dataset_modifie.columns

    #nombre de valeur dans le dataset
    Nbr_values=dataset_modifie.shape[0]*dataset_modifie.shape[1]
    
    #nombre de valeur a supprimer
    delete=int(Nbr_values*pourcentage/100)
    
    while(pd.isnull(dataset_modifie).sum().sum()<delete):
        #selection de l'index
        index_delete=random.randrange(Nbr_values)
        columns_delete=random.choice(columns)
        dataset_modifie[columns_delete][index_delete]= np.NaN
        
    return dataset_modifie

In [8]:
def analyse_des_cas_concret(dataset):
    dataset_modifie=dataset
    index_with_nan = dataset_modifie.index[dataset_modifie.isnull().any(axis=1)]
    dataset_modifie.drop(index_with_nan,0, inplace=True)
    
    return dataset_modifie

In [9]:
def Analyse_des_données_disponibles(taux,dataset):
    index=[]
    
    taux=int(taux)
    
    if taux >len(dataset.columns) or taux<0:
        #pour que le taux soit compris entre 
        print(f"Le taux doit etre inférieure a {dataset.columns} et supérieure a 0")
        exit(-1)
   #tableau calculan les index des nan pour chaque colonne du dataframe et les concatenant dans une liste     
    for i in dataset.columns:
        indexcolumns=dataset.index[pd.isnull(df[i])]
        if indexcolumns.shape !=0:
            for cmpt in range(len(indexcolumns)):
                index.append(indexcolumns[cmpt])
                
    #récuperation des indices et du nombre d'iteration de chaque indices
    index_ligne=np.unique(index,return_counts=True)[0]
    number_nan=np.unique(index,return_counts=True)[1]
    
    nan_sup_taux=np.where(number_nan>=taux)[0]
    
    #liste associant les array superieur aux taux de np.unique aux array réel
    index_delete=[]
    for i in nan_sup_taux:
        index_delete.append(index_ligne[i])
    
    
    #suppresion des ligne
    dataset.drop(index_delete,0,inplace=True)
    
    return dataset

In [10]:
def Analyse_des_données_disponibles_V2(pourcentage,dataset):
    index=[]
    pourcentage=int(pourcentage)
    dataset_modifie=dataset.copy()
    columns=dataset_modifie.columns
    #vérification pour chaque colonne
    for i in columns:
        #nombre de nan dans la colonne
        number_of_nan=dataset_modifie[i].isnull().sum()
        
       #si nombre de nan/ taille de la taille superieur ou egale au taux on supp
        if ((number_of_nan / dataset_modifie[i].shape[0])>=(pourcentage/100)) :
            dataset_modifie=dataset_modifie.drop(i, axis=1)
    return dataset_modifie
        
        

In [8]:
def score_estimator_LR(name_columns_X,name_columns_Y,dataset,numbereofImputerchoosen):
    
    ######numbereofImputerchoosen####
    
    #1:SimpleImputer(missing_values=np.nan, strategy='mean')
    #2:SimpleImputer(missing_values=np.nan, strategy='median')
    #3:IterativeImputer(estimator=BayesianRidge(), n_nearest_features=None, imputation_order='ascending')
    #4:KNNImputer(n_neighbors=2)

    
    datet_modifie=dataset.copy()
    
    #définition du X et du Y
    X= datet_modifie[name_columns_X]
    y= datet_modifie[name_columns_Y]
    
    
    #Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state =0)
    
    #Création des df train et test
    dataframe_train=pd.concat([X_train, y_train],axis=1)
    dataframe_test=pd.concat([X_test, y_test],axis=1)
    
    
    #IMPUTATION DES DONNEE
    
    # definition du modele imputer
    if numbereofImputerchoosen==1:
        imputer=SimpleImputer(missing_values=np.nan, strategy='mean')
    elif numbereofImputerchoosen==2:
        imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    elif numbereofImputerchoosen==3:
        imputer = IterativeImputer(estimator=BayesianRidge(), n_nearest_features=None, imputation_order='ascending')
    elif numbereofImputerchoosen==4:
        imputer = KNNImputer(n_neighbors=5)
    
    
    
    
    cmpt=0
    for i in [dataframe_train,dataframe_test]:
        buffer=i
        #print(f"avant imputation :{buffer.isnull().sum()}")
        # fit sur le dataset
        imputer.fit(buffer)
        # transform du dataset
        buffer = imputer.transform(buffer)
        #on retransforme en df
        buffer = pd.DataFrame(buffer, columns = [name_columns_X,name_columns_Y])
        #print(f"Apres imputation :{buffer.isnull().sum()}")
        
        if cmpt == 0:
            dataframe_train=buffer.copy()
        else:
            dataframe_test=buffer.copy()
        cmpt=cmpt+1
        
    #print(dataframe_train.isnull().sum())        
#recuperation des X_train,y_train,X_test,y_test concatené
    X_train=dataframe_train[name_columns_X]
    y_train=dataframe_train[name_columns_Y]
    X_test=dataframe_test[name_columns_X]
    y_test=dataframe_test[name_columns_Y]
    #remise de series --> DF
    X_train=pd.DataFrame(data=X_train)
    y_train=pd.DataFrame(data=y_train)
    X_test=pd.DataFrame(data=X_test)
    y_test=pd.DataFrame(data=y_test)
    
    
#entrainement 
    model = LinearRegression().fit(X_train, y_train)
    score_train=model.score(X_train, y_train)
    #print(f"le score du model est de :{model.score(X_train, y_train)}")
    #print(f"Coef de la regresssion :{model.coef_[0][0]}")
    #print(f"Intercept du modele: {model.intercept_[0]}")
#test
    y_predict = model.predict(X_test)
    model_mse = mean_squared_error(y_predict, y_test) 
    #print(model_mse)
    #print(model.score(X_test, y_test))
    score_test=model.score(X_test, y_test)
    
    return score_train,score_test,model_mse



In [7]:
def score_estimator_LR_v2(name_columns_X,name_columns_Y,dataset,numbereofImputerchoosen):
    
    ######numbereofImputerchoosen####
    
    #1:SimpleImputer(missing_values=np.nan, strategy='mean')
    #2:SimpleImputer(missing_values=np.nan, strategy='median')
    #3:IterativeImputer(estimator=BayesianRidge(), n_nearest_features=None, imputation_order='ascending')
    #4:KNNImputer(n_neighbors=2)

    
    datet_modifie=dataset.copy()
    
    #définition du X et du Y
    #["ubi","hbui","vu"]
    X= datet_modifie[name_columns_X]
    y= datet_modifie[name_columns_Y]
    
    
    
    
    #Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state =0)
    
    #Création des df train et test
    dataframe_train=pd.concat([X_train, y_train],axis=1)
    dataframe_test=pd.concat([X_test, y_test],axis=1)
  
    
    #IMPUTATION DES DONNEE
    
    # definition du modele imputer
    if numbereofImputerchoosen==1:
        imputer=SimpleImputer(missing_values=np.nan, strategy='mean')
    elif numbereofImputerchoosen==2:
        imputer = SimpleImputer(missing_values=np.nan, strategy='median')
    elif numbereofImputerchoosen==3:
        imputer = IterativeImputer(estimator=BayesianRidge(), n_nearest_features=None, imputation_order='ascending')
    elif numbereofImputerchoosen==4:
        imputer = KNNImputer(n_neighbors=5)
    
    
    
    columns=dataframe_train.columns
    cmpt=0
    for i in [dataframe_train,dataframe_test]:
        buffer=i
        #print(f"avant imputation :{buffer.isnull().sum()}")
        # fit sur le dataset
        imputer.fit(buffer)
        # transform du dataset
        buffer = imputer.transform(buffer)
        #on retransforme en df
        buffer = pd.DataFrame(buffer, columns = [columns])
        #print(f"Apres imputation :{buffer.isnull().sum()}")
        
        if cmpt == 0:
            dataframe_train=buffer.copy()
        else:
            dataframe_test=buffer.copy()
        cmpt=cmpt+1
        
    #print(dataframe_train.isnull().sum())        
#recuperation des X_train,y_train,X_test,y_test concatené
    X_train=dataframe_train[name_columns_X]
    y_train=dataframe_train[name_columns_Y]
    X_test=dataframe_test[name_columns_X]
    y_test=dataframe_test[name_columns_Y]
    #remise de series --> DF
    X_train=pd.DataFrame(data=X_train)
    y_train=pd.DataFrame(data=y_train)
    X_test=pd.DataFrame(data=X_test)
    y_test=pd.DataFrame(data=y_test)
    
    
#entrainement 
    model = LinearRegression().fit(X_train, y_train)
    score_train=model.score(X_train, y_train)
    #print(f"le score du model est de :{model.score(X_train, y_train)}")
    #print(f"Coef de la regresssion :{model.coef_[0][0]}")
    #print(f"Intercept du modele: {model.intercept_[0]}")
#test
    y_predict = model.predict(X_test)
    model_mse = mean_squared_error(y_predict, y_test) 
    #print(model_mse)
    #print(model.score(X_test, y_test))
    score_test=model.score(X_test, y_test)
    
    return score_train,score_test,model_mse

In [27]:
def concat_tab(list1,list2,list3,list4,colum_name_list1,colum_name_list2):
    
    #creation d'une colonne imputer name pour les df
    imputer_name=["SimpleImputer_MEAN","SimpleImputer_MEDIAN","IterativeImputer","KNNImputer"]
    df_imputer_name=pd.DataFrame(data=imputer_name)
    df_imputer_name.columns=["imputer_name"]

    #list in tab
    #TRAIN
    #LIST1
    dflist1=pd.DataFrame(data=list1)
    dflist1.columns=[colum_name_list1]
    #list in tab
    #TEST
    #LIST2
    dflist2=pd.DataFrame(data=list2)
    dflist2.columns=[colum_name_list1]
    #list in tab
    #TRAIN
    #LIST3
    dflist3=pd.DataFrame(data=list3)
    dflist3.columns=[colum_name_list2]
    #list in tab
    #TEST
    #LIST4
    dflist4=pd.DataFrame(data=list4)
    dflist4.columns=[colum_name_list2]
    
    #tabtrain list1 and liste 3
    dftrain=pd.DataFrame(data=['train','train','train','train'])
    dftrain.columns=["model"]
    df_result_train=pd.concat([df_imputer_name,dftrain,dflist1, dflist3],axis=1)
    
    
    #tabtest list2 and list4
    dftest=pd.DataFrame(data=['test','test','test','test'])
    dftest.columns=["model"]
    df_result_test=pd.concat([df_imputer_name,dftest,dflist2, dflist4],axis=1)
    
    
    #test and train
    df_result=pd.concat([df_result_train,df_result_test],axis=0)
 
    return df_result